# 🧮 Phase 2B — Multi-year Bioeconomic LP with Dynamics
This notebook adds:
1) **Crop storage carryover** (inventory `inv[h,c,t]`)
2) **Livestock herd dynamics** (herd `herd[h,l,t]` and sales `sale_live[h,l,t]`)

Still linear (LP), solved with `scipy.optimize.linprog`.


In [ ]:
# ========================================
# SETUP: IMPORT LIBRARIES AND PATHS
# ========================================
# Import the tools we need for calculations and data handling

import json                        # For reading scenario configuration files
from dataclasses import dataclass  # For creating data structure templates
from typing import Dict            # For type hints (code documentation)
from pathlib import Path           # For handling file paths

import numpy as np                 # For numerical calculations (arrays, math)
import pandas as pd                # For working with data tables (like Excel)
from scipy.optimize import linprog # The optimization solver that finds best decisions

# Try to import display tool for pretty tables
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

# Define where our input data and outputs are stored
DATA = Path('./data_phase_2B')  # Folder with CSV files for Phase 2B
OUT = Path('./outputs')          # Folder where results will be saved
OUT.mkdir(exist_ok=True, parents=True)  # Create outputs folder if it doesn't exist

print('Loaded numpy, pandas, scipy.optimize.linprog')


Loaded numpy, pandas, scipy.optimize.linprog


In [ ]:
# ========================================
# LOAD AND PREVIEW INPUT DATA
# ========================================
# Read all CSV files and scenario configuration, including NEW initial conditions

# Helper function to check if required files exist
def must(path: Path) -> Path:
    """
    Checks if a file exists and raises an error if missing
    
    INPUTS: path - Path to a required file
    OUTPUTS: Returns the same path if file exists
    PURPOSE: Prevents the model from running with missing data
    """
    if not path.exists():
        raise FileNotFoundError(f'Missing required file: {path}')
    return path

# Load baseline data CSVs (same as Phase 2A)
hh_df = pd.read_csv(must(DATA/'households.csv'))      # Household characteristics
crops_df = pd.read_csv(must(DATA/'crops.csv'))        # Crop parameters (WITH storage fields)
livest_df = pd.read_csv(must(DATA/'livestock.csv'))   # Livestock parameters (WITH reproduction)
prices_df = pd.read_csv(must(DATA/'prices.csv'))      # Market prices (wages)

# Load the scenario configuration (multi-year settings)
scenario = json.load(open(must(DATA/'scenario_multi.json'),'r'))

# NEW: Load initial conditions for dynamic simulations
# These define the starting point (year 0) before optimization begins
init_stocks_df = pd.read_csv(must(DATA/'initial_stocks.csv'))  # Starting crop inventory
init_herd_df = pd.read_csv(must(DATA/'initial_herd.csv'))      # Starting livestock herd size

# Preview all the loaded data
# This lets you see what data the model will use
for name, df in [
    ('households.csv', hh_df),
    ('crops.csv', crops_df),
    ('livestock.csv', livest_df),
    ('prices.csv', prices_df),
    ('initial_stocks.csv', init_stocks_df),      # NEW: Starting crop inventory
    ('initial_herd.csv', init_herd_df),          # NEW: Starting livestock
    ('scenario_multi.json', pd.DataFrame([{'T':scenario.get('T'), 'discount_rate':scenario.get('discount_rate')}])),
]:
    if display_dataframe_to_user: 
        display_dataframe_to_user(name, df)
    else: 
        display(df.head())

print('Loaded inputs.')


,name,n_households,adult_equiv,labor_endowment,land_available,max_hired_labor
0,HFR,100,3.8,220,0.8,120
1,HMR,100,3.9,240,1.0,130
2,MFIR,100,4.2,300,1.2,160
3,MMR,100,4.2,300,1.2,160
4,MMIR,100,4.6,320,1.5,180


,name,calorie_per_kg,yield_per_ha,price_sale,seed_cost_per_ha,fert_cost_per_ha,chem_cost_per_ha,labor_req_per_ha,storage_cost_per_kg,max_storage_kg,spoilage_rate
0,maize,3600,3500,5.0,800,1800,600,60,1,500,0.05
1,wheat,3200,3000,6.0,700,1600,500,50,1,400,0.04
2,rice,4000,4000,7.0,900,2000,700,80,1,600,0.03
3,beans,3400,1500,8.0,900,700,400,70,2,300,0.06


,name,price_sale,feed_cost_per_unit,vet_cost_per_unit,labor_req_per_unit,reproduction_rate,max_herd
0,goat,2500,800,200,6.0,0.10,100
1,chicken,300,60,15,0.6,0.15,200


,wage
0,80


,household_class,crop,stock_kg
0,HFR,maize,0.0


,household_class,livestock,herd_units
0,HFR,goat,0.0


,T,discount_rate
0,10,0.1


Loaded inputs.


In [ ]:
# ========================================
# DATA STRUCTURES (BLUEPRINTS) - ENHANCED FOR DYNAMICS
# ========================================
# Similar to Phase 2A, but with NEW fields for storage and livestock dynamics

@dataclass
class HouseholdClass:
    """
    Blueprint for a household group - same as Phase 2A
    Stores resources available: land, labor, household size, etc.
    """
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float

@dataclass
class CropParam:
    """
    Blueprint for a crop type - ENHANCED with storage parameters
    
    NEW FIELDS (compared to Phase 2A):
    - storage_cost_per_kg: Annual cost to store 1 kg (warehousing, preservation)
    - max_storage_kg: Maximum storage capacity (warehouse limit)
    - spoilage_rate: Fraction lost during storage (0.05 = 5% spoilage per year)
    
    PURPOSE: Allows farmers to store crops and sell later when prices are better
    Example: Store maize after harvest, sell during lean season at higher prices
    Trade-off: Storage costs money and some crops spoil
    """
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float
    storage_cost_per_kg: float    # NEW: Cost to store per kg per year
    max_storage_kg: float          # NEW: Maximum storage capacity
    spoilage_rate: float           # NEW: Fraction lost (0.05 = 5% spoilage)

@dataclass
class LivestockParam:
    """
    Blueprint for livestock - ENHANCED with herd dynamics
    
    NEW FIELDS (compared to Phase 2A):
    - reproduction_rate: Natural herd growth rate (0.15 = 15% growth per year from births)
    - max_herd: Maximum herd size (grazing land limit, shelter capacity)
    
    PURPOSE: Livestock reproduce naturally - herd grows over time
    Example: Start with 10 cattle, reproduction_rate=0.15 → 11.5 cattle next year
    Can sell offspring for income or keep them to grow herd
    Trade-off: Larger herd = more income potential, but also higher feed/vet costs
    """
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float
    reproduction_rate: float       # NEW: Natural herd growth rate
    max_herd: float                # NEW: Maximum herd capacity

@dataclass
class PriceParam:
    """
    Blueprint for market prices - same as Phase 2A
    Currently just the wage rate
    """
    wage: float

@dataclass
class ModelParams:
    """
    Master container that holds ALL baseline data
    Same structure as Phase 2A
    """
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

# ========================================
# FUNCTION: VALIDATE AND LOAD PARAMETERS
# ========================================

def _req(x, field):
    """
    Helper function to validate that required fields are not missing
    
    INPUTS: 
    - x: The value to check
    - field: Name of the field (for error messages)
    
    OUTPUTS: Returns the value as a float if valid
    PURPOSE: Catches missing/invalid data before running the model
    """
    if pd.isna(x) or str(x).strip()=='':
        raise ValueError(f'Missing value for {field}')
    return float(x)

def load_params() -> ModelParams:
    """
    Reads CSV files and creates structured parameter objects
    Now includes storage and reproduction parameters
    
    INPUTS: Uses global dataframes (hh_df, crops_df, livest_df, prices_df)
    OUTPUTS: Returns ModelParams object with all baseline data
    
    HOW IT WORKS:
    1. Loop through each CSV row
    2. Validate all required fields exist
    3. Create data structure objects (with NEW dynamic fields)
    4. Bundle everything into ModelParams
    """
    # Load households with validation (same as Phase 2A)
    households = {str(r['name']).strip(): HouseholdClass(
        name=str(r['name']).strip(),
        n_households=_req(r['n_households'],'households.n_households'),
        adult_equiv=_req(r['adult_equiv'],'households.adult_equiv'),
        labor_endowment=_req(r['labor_endowment'],'households.labor_endowment'),
        land_available=_req(r['land_available'],'households.land_available'),
        max_hired_labor=_req(r['max_hired_labor'],'households.max_hired_labor'),
    ) for _, r in hh_df.iterrows()}

    # Load crops with validation (NOW with storage parameters)
    crops = {str(r['name']).strip(): CropParam(
        name=str(r['name']).strip(),
        calorie_per_kg=_req(r['calorie_per_kg'],'crops.calorie_per_kg'),
        yield_per_ha=_req(r['yield_per_ha'],'crops.yield_per_ha'),
        price_sale=_req(r['price_sale'],'crops.price_sale'),
        seed_cost_per_ha=_req(r['seed_cost_per_ha'],'crops.seed_cost_per_ha'),
        fert_cost_per_ha=_req(r['fert_cost_per_ha'],'crops.fert_cost_per_ha'),
        chem_cost_per_ha=_req(r['chem_cost_per_ha'],'crops.chem_cost_per_ha'),
        labor_req_per_ha=_req(r['labor_req_per_ha'],'crops.labor_req_per_ha'),
        storage_cost_per_kg=_req(r['storage_cost_per_kg'],'crops.storage_cost_per_kg'),  # NEW
        max_storage_kg=_req(r['max_storage_kg'],'crops.max_storage_kg'),                  # NEW
        spoilage_rate=_req(r['spoilage_rate'],'crops.spoilage_rate'),                    # NEW
    ) for _, r in crops_df.iterrows()}

    # Load livestock with validation (NOW with reproduction and herd limits)
    livestock = {str(r['name']).strip(): LivestockParam(
        name=str(r['name']).strip(),
        price_sale=_req(r['price_sale'],'livestock.price_sale'),
        feed_cost_per_unit=_req(r['feed_cost_per_unit'],'livestock.feed_cost_per_unit'),
        vet_cost_per_unit=_req(r['vet_cost_per_unit'],'livestock.vet_cost_per_unit'),
        labor_req_per_unit=_req(r['labor_req_per_unit'],'livestock.labor_req_per_unit'),
        reproduction_rate=_req(r['reproduction_rate'],'livestock.reproduction_rate'),    # NEW
        max_herd=_req(r['max_herd'],'livestock.max_herd'),                               # NEW
    ) for _, r in livest_df.iterrows()}

    # Load prices
    prices = PriceParam(wage=_req(prices_df.iloc[0]['wage'],'prices.wage'))
    
    return ModelParams(households=households, crops=crops, livestock=livestock, prices=prices)

# ========================================
# FUNCTION: LOAD MULTI-YEAR SCENARIO
# ========================================

def _as_T(x, T, name):
    """
    Converts scenario parameters to T-length arrays (one value per year)
    Same as Phase 2A
    """
    if isinstance(x, (int,float)): 
        return [float(x)]*T
    if isinstance(x, list):
        if len(x)!=T: 
            raise ValueError(f'scenario.{name} must have length T={T}, got {len(x)}')
        return [float(v) for v in x]
    raise ValueError(f'scenario.{name} must be number or length-T list')

def load_scenario(sc):
    """
    Loads multi-year scenario with time-varying multipliers and discount factors
    Same as Phase 2A
    
    OUTPUTS: Returns 8 values:
    - T: Number of years to simulate
    - r: Discount rate
    - Y: Yield multipliers per year
    - P: Price multipliers per year
    - W: Wage multipliers per year
    - F: Fertilizer price multipliers per year
    - POP: Population multipliers per year
    - DISC: Discount factors per year (for NPV calculation)
    """
    T = int(sc.get('T', 10))
    r = float(sc.get('discount_rate', 0.10))
    Y = _as_T(sc.get('yield_multiplier', 1.0), T, 'yield_multiplier')
    P = _as_T(sc.get('crop_price_multiplier', 1.0), T, 'crop_price_multiplier')
    W = _as_T(sc.get('wage_multiplier', 1.0), T, 'wage_multiplier')
    F = _as_T(sc.get('fert_price_multiplier', 1.0), T, 'fert_price_multiplier')
    POP = _as_T(sc.get('population_multiplier', 1.0), T, 'population_multiplier')
    DISC = [1.0/((1.0+r)**t) for t in range(T)]
    return T, r, Y, P, W, F, POP, DISC

# ========================================
# EXECUTE: LOAD ALL PARAMETERS
# ========================================
# Load baseline parameters from CSVs
params = load_params()

# Load multi-year scenario arrays
T, r, Ymul, Pmul, Wmul, Fmul, POPmul, DISC = load_scenario(scenario)

# Display what we loaded
print(f'Loaded: H={len(params.households)}, C={len(params.crops)}, L={len(params.livestock)}, T={T}, r={r}')


Loaded: H=6, C=4, L=2, T=10, r=0.1


In [ ]:
# ========================================
# FUNCTION: BUILD VARIABLE INDEX MAP (WITH DYNAMICS)
# ========================================
def build_index(H, C, L, T):
    """
    Creates mapping for ALL decision variables across ALL years
    Like Phase 2A, but with NEW variables for storage and herd management
    
    INPUTS:
    - H: List of household groups
    - C: List of crops
    - L: List of livestock types
    - T: Number of years
    
    OUTPUTS:
    - idx: Dictionary mapping (decision type, household, item, year) → position number
    - pos: Total number of decision variables
    
    NEW VARIABLES (compared to Phase 2A):
    - 'inv' (inventory): How much crop to store at end of year (kg)
    - 'herd': How many animals to keep in the herd (not sell)
    - 'sale_live': How many animals to sell this year
    
    REMOVED VARIABLES (compared to Phase 2A):
    - 'stored': No longer needed - replaced by dynamic 'inv' variable
    - 'live_units': Split into 'herd' (keep) and 'sale_live' (sell)
    
    PURPOSE: Track dynamic decisions - storage and herd size evolve over time
    """
    idx = {}  # Dictionary to store mappings
    pos = 0   # Counter for variable positions
    
    # For each year, create variables for all decisions
    for t in range(T):
        # LAND ALLOCATION per crop per household per year
        for h in H:
            for c in C: 
                idx[('area', h, c, t)] = pos
                pos += 1
        
        # CONSUMPTION per crop per household per year
        for h in H:
            for c in C: 
                idx[('cons', h, c, t)] = pos
                pos += 1
        
        # SALES per crop per household per year
        for h in H:
            for c in C: 
                idx[('sold', h, c, t)] = pos
                pos += 1
        
        # INVENTORY (STORAGE) per crop per household per year [NEW]
        # This is the amount stored at the END of year t, available for next year
        for h in H:
            for c in C: 
                idx[('inv', h, c, t)] = pos
                pos += 1
        
        # HIRED LABOR per household per year
        for h in H: 
            idx[('hired', h, None, t)] = pos
            pos += 1
        
        # OFF-FARM WORK per household per year
        for h in H: 
            idx[('off_farm', h, None, t)] = pos
            pos += 1
        
        # HERD SIZE per livestock per household per year [NEW]
        # Number of animals KEPT in the herd (not sold)
        for h in H:
            for l in L: 
                idx[('herd', h, l, t)] = pos
                pos += 1
        
        # LIVESTOCK SALES per type per household per year [NEW]
        # Number of animals SOLD this year
        for h in H:
            for l in L: 
                idx[('sale_live', h, l, t)] = pos
                pos += 1
    
    return idx, pos

# ========================================
# FUNCTION: LOAD INITIAL CONDITIONS
# ========================================
def init_maps(init_stocks_df, init_herd_df, H, C, L):
    """
    Creates dictionaries of initial conditions (year 0 state)
    
    INPUTS:
    - init_stocks_df: CSV with starting crop inventory per household
    - init_herd_df: CSV with starting livestock herd per household
    - H, C, L: Lists of households, crops, livestock
    
    OUTPUTS:
    - init_stock: Dict mapping (household, crop) → initial kg stored
    - init_herd: Dict mapping (household, livestock) → initial herd size
    
    PURPOSE: Defines the starting point for dynamic simulations
    Example: "Poor households start with 500 kg maize stored and 5 cattle"
    
    If a combination is not in the CSV, defaults to 0.0
    """
    # Load initial crop stocks from CSV
    init_stock = {(str(r['household_class']).strip(), str(r['crop']).strip()): float(r['stock_kg'])
                  for _, r in init_stocks_df.iterrows()}
    
    # Load initial livestock herd from CSV
    init_herd = {(str(r['household_class']).strip(), str(r['livestock']).strip()): float(r['herd_units'])
                 for _, r in init_herd_df.iterrows()}
    
    # Fill in missing combinations with zero
    for h in H:
        for c in C: 
            init_stock.setdefault((h,c), 0.0)
        for l in L: 
            init_herd.setdefault((h,l), 0.0)
    
    return init_stock, init_herd

# ========================================
# FUNCTION: SOLVE MULTI-YEAR OPTIMIZATION WITH DYNAMICS
# ========================================
def solve_phase2B(params: ModelParams, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC):
    """
    Solves the MULTI-YEAR farm optimization with DYNAMIC LINKAGES between years
    
    KEY DIFFERENCES FROM PHASE 2A:
    1. CROP STORAGE: Can store crops to sell later or eat later
       - Inventory from year t carries over to year t+1 (minus spoilage)
       - Storage costs money but provides flexibility
    
    2. LIVESTOCK HERD DYNAMICS: Animals reproduce naturally
       - Herd grows each year by reproduction_rate
       - Can sell offspring or keep them to grow herd
       - Must pay feed/vet costs for entire herd
    
    3. INTER-YEAR LINKAGES: Years are now connected!
       - Inventory[t] + Production[t+1] = Consumption[t+1] + Sales[t+1] + Inventory[t+1]
       - Herd[t] × (1 + reproduction) = Herd[t+1] + Sales[t+1]
    
    INPUTS: Same as Phase 2A, plus uses global init_stocks_df and init_herd_df
    OUTPUTS: Same as Phase 2A (res, idx, H, C, L)
    
    HOW IT WORKS:
    The objective is still NPV (like Phase 2A), but now the model can:
    - Store crops when prices are low, sell when prices are high
    - Build up livestock herds over time for future income
    - Balance short-term profit vs long-term wealth accumulation
    """
    
    # Step 1: Setup
    H = list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())
    idx, nvars = build_index(H, C, L, T)
    cvec = np.zeros(nvars)
    bounds = [(0, None) for _ in range(nvars)]
    
    # Load initial conditions (starting inventory and herd sizes)
    init_stock, init_herd = init_maps(init_stocks_df, init_herd_df, H, C, L)

    # Step 2: Build OBJECTIVE FUNCTION (maximize NPV)
    # Similar to Phase 2A, but with NEW costs/revenues for storage and livestock sales
    for t in range(T):
        disc = DISC[t]  # Discount factor
        wage_t = params.prices.wage * Wmul[t]
        
        for h in H:
            # COSTS
            cvec[idx[('hired', h, None, t)]] += disc * wage_t
            
            # REVENUES
            cvec[idx[('off_farm', h, None, t)]] += -disc * wage_t
            
            for cn in C:
                cp = params.crops[cn]
                # Revenue from crop sales
                cvec[idx[('sold', h, cn, t)]] += -disc * (cp.price_sale * Pmul[t])
                
                # Cost of crop inputs
                per_ha = cp.seed_cost_per_ha + cp.fert_cost_per_ha * Fmul[t] + cp.chem_cost_per_ha
                cvec[idx[('area', h, cn, t)]] += disc * per_ha
                
                # NEW: Cost of storing crops
                cvec[idx[('inv', h, cn, t)]] += disc * cp.storage_cost_per_kg
            
            for l in L:
                lv = params.livestock[l]
                # NEW: Revenue from livestock sales (separate from herd keeping)
                cvec[idx[('sale_live', h, l, t)]] += -disc * lv.price_sale
                
                # NEW: Cost of maintaining herd (feed + vet for animals we keep)
                cvec[idx[('herd', h, l, t)]] += disc * (lv.feed_cost_per_unit + lv.vet_cost_per_unit)

    # Step 3: Initialize constraint matrices
    A_eq, b_eq = [], []  # Equality constraints (dynamic balances)
    A_ub, b_ub = [], []  # Inequality constraints (limits)

    # ========================================
    # CONSTRAINT 1: INVENTORY DYNAMICS (NEW!)
    # ========================================
    # This links years together through crop storage
    # Balance equation: Inventory[t] = Inventory[t-1] × (1-spoilage) + Production[t] - Consumption[t] - Sales[t]
    #
    # Think of it like a bank account:
    # - Starting balance (from last year, minus spoilage)
    # - Deposits (new harvest)
    # - Withdrawals (eating and selling)
    # - Ending balance (stored for next year)
    
    for t in range(T):
        for h in H:
            for cn in C:
                cp = params.crops[cn]
                row = np.zeros(nvars)
                
                # Left side: Inventory at end of year t
                row[idx[('inv', h, cn, t)]] = 1.0
                
                # Right side: Production this year
                row[idx[('area', h, cn, t)]] = -(cp.yield_per_ha * Ymul[t])
                
                # Right side: Consumption this year
                row[idx[('cons', h, cn, t)]] = 1.0
                
                # Right side: Sales this year
                row[idx[('sold', h, cn, t)]] = 1.0
                
                if t == 0:
                    # First year: Start with initial stock (minus spoilage)
                    A_eq.append(row)
                    b_eq.append((1.0 - cp.spoilage_rate) * init_stock[(h, cn)])
                else:
                    # Later years: Start with last year's inventory (minus spoilage)
                    row[idx[('inv', h, cn, t-1)]] = -(1.0 - cp.spoilage_rate)
                    A_eq.append(row)
                    b_eq.append(0.0)

    # ========================================
    # CONSTRAINT 2: HERD DYNAMICS (NEW!)
    # ========================================
    # This links years together through livestock reproduction
    # Balance equation: Herd[t] + Sales[t] = Herd[t-1] × (1 + reproduction_rate)
    #
    # Think of it like a growing population:
    # - Last year's herd reproduces (births)
    # - Decide how many to keep vs sell
    # - Remaining herd goes into next year
    #
    # Example: 10 cattle with 15% reproduction = 11.5 cattle available
    # Can keep 10 and sell 1.5, or keep all 11.5, or sell some of the original herd
    
    for t in range(T):
        for h in H:
            for l in L:
                lv = params.livestock[l]
                row = np.zeros(nvars)
                
                # Left side: Herd kept + Animals sold
                row[idx[('herd', h, l, t)]] = 1.0
                row[idx[('sale_live', h, l, t)]] = 1.0
                
                if t == 0:
                    # First year: Start with initial herd (plus natural reproduction)
                    A_eq.append(row)
                    b_eq.append((1.0 + lv.reproduction_rate) * init_herd[(h, l)])
                else:
                    # Later years: Last year's herd reproduces
                    row[idx[('herd', h, l, t-1)]] = -(1.0 + lv.reproduction_rate)
                    A_eq.append(row)
                    b_eq.append(0.0)

    # ========================================
    # CONSTRAINT 3: LAND AVAILABILITY
    # ========================================
    # Same as Phase 2A
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C:
                row[idx[('area', h, cn, t)]] = 1.0
            A_ub.append(row)
            b_ub.append(params.households[h].land_available)

    # ========================================
    # CONSTRAINT 4: LABOR AVAILABILITY
    # ========================================
    # Now uses 'herd' instead of 'live_units'
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            # Crop labor
            for cn in C: 
                row[idx[('area', h, cn, t)]] = params.crops[cn].labor_req_per_ha
            # Livestock labor (for herd we keep, not what we sell)
            for l in L: 
                row[idx[('herd', h, l, t)]] = params.livestock[l].labor_req_per_unit
            # Off-farm and hired labor
            row[idx[('off_farm', h, None, t)]] = 1.0
            row[idx[('hired', h, None, t)]] = -1.0
            A_ub.append(row)
            b_ub.append(params.households[h].labor_endowment)

    # ========================================
    # CONSTRAINT 5: HIRED LABOR LIMIT
    # ========================================
    # Same as Phase 2A
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            row[idx[('hired', h, None, t)]] = 1.0
            A_ub.append(row)
            b_ub.append(params.households[h].max_hired_labor)

    # ========================================
    # CONSTRAINT 6: NUTRITION REQUIREMENT
    # ========================================
    # Same as Phase 2A
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C: 
                row[idx[('cons', h, cn, t)]] = -params.crops[cn].calorie_per_kg
            kcal_need = params.min_kcal_per_person_per_day * (params.households[h].adult_equiv * POPmul[t]) * params.days_per_year
            A_ub.append(row)
            b_ub.append(-kcal_need)

    # ========================================
    # CONSTRAINT 7: STORAGE CAPACITY (NEW!)
    # ========================================
    # Inventory cannot exceed warehouse/storage capacity
    for t in range(T):
        for h in H:
            for cn in C:
                cp = params.crops[cn]
                row = np.zeros(nvars)
                row[idx[('inv', h, cn, t)]] = 1.0
                A_ub.append(row)
                b_ub.append(cp.max_storage_kg)

    # ========================================
    # CONSTRAINT 8: HERD CAPACITY (NEW!)
    # ========================================
    # Herd size cannot exceed grazing land or shelter capacity
    for t in range(T):
        for h in H:
            for l in L:
                lv = params.livestock[l]
                if lv.max_herd and lv.max_herd > 0:
                    row = np.zeros(nvars)
                    row[idx[('herd', h, l, t)]] = 1.0
                    A_ub.append(row)
                    b_ub.append(lv.max_herd)

    # Step 4: SOLVE the optimization
    res = linprog(cvec, 
                  A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  bounds=bounds, 
                  method='highs')
    
    return res, idx, H, C, L

# ========================================
# EXECUTE: SOLVE AND EXTRACT RESULTS
# ========================================

# Step 1: Solve the optimization
res, idx, H, C, L = solve_phase2B(params, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC)

# Step 2: Check if optimization succeeded
print('Status:', res.message)
print('Success:', bool(res.success))
if not res.success:
    raise RuntimeError('Optimization failed. Check feasibility / coefficients.')

# Step 3: Extract solution
x = res.x

# Helper function to get values
def v(kind, h, k, t): 
    """Get the optimal value for a decision variable"""
    return float(x[idx[(kind, h, k, t)]])

# Step 4: Calculate detailed results for each year and household
rows=[]
for t in range(T):
    wage_t = params.prices.wage * Wmul[t]
    
    for h in H:
        # REVENUES
        rev_crops = sum(v('sold', h, cn, t) * (params.crops[cn].price_sale * Pmul[t]) for cn in C)
        rev_livestock = sum(v('sale_live', h, l, t) * params.livestock[l].price_sale for l in L)
        rev_off = v('off_farm', h, None, t) * wage_t
        
        # COSTS
        cost_crop_inputs = sum(v('area', h, cn, t) * 
                              (params.crops[cn].seed_cost_per_ha + 
                               params.crops[cn].fert_cost_per_ha * Fmul[t] + 
                               params.crops[cn].chem_cost_per_ha) 
                              for cn in C)
        cost_storage = sum(v('inv', h, cn, t) * params.crops[cn].storage_cost_per_kg for cn in C)  # NEW
        cost_livestock = sum(v('herd', h, l, t) * 
                            (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) 
                            for l in L)
        cost_hired = v('hired', h, None, t) * wage_t
        
        # PROFIT
        profit = (rev_crops + rev_livestock + rev_off) - (cost_crop_inputs + cost_storage + cost_livestock + cost_hired)
        
        # Store results
        rows.append({
            'year': t+1, 
            'household_class': h,
            'rev_crops': rev_crops, 
            'rev_livestock_sales': rev_livestock,  # Now specifically sales
            'rev_off_farm': rev_off,
            'cost_crop_inputs': cost_crop_inputs, 
            'cost_storage': cost_storage,  # NEW
            'cost_livestock': cost_livestock, 
            'cost_hired_labor': cost_hired,
            'profit': profit, 
            'discount_factor': DISC[t], 
            'discounted_profit': profit*DISC[t]
        })

# Step 5: Create yearly results table
yearly_df = pd.DataFrame(rows)

# Step 6: Calculate NPV summary
npv_df = yearly_df.groupby('household_class', as_index=False).agg(NPV=('discounted_profit','sum'))

# Step 7: Display results
if display_dataframe_to_user:
    display_dataframe_to_user('Phase 2B — Yearly results', yearly_df)
    display_dataframe_to_user('Phase 2B — NPV summary', npv_df)
else:
    display(yearly_df.head(12))
    display(npv_df)

# Step 8: Save results
yearly_df.to_csv(OUT/'phase2B_yearly_results.csv', index=False)
npv_df.to_csv(OUT/'phase2B_npv_summary.csv', index=False)
json.dump({
    'success': True, 
    'objective_npv': float(-res.fun), 
    'T': T, 
    'discount_rate': r
}, open(OUT/'phase2B_meta.json','w'), indent=2)

print('Saved outputs in ./outputs/')


Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True


,year,household_class,rev_crops,rev_livestock_sales,rev_off_farm,cost_crop_inputs,cost_storage,cost_livestock,cost_hired_labor,profit,discount_factor,discounted_profit
0,1,HFR,17545.50,0.0,22080.0,2880.0,0.0,0.0,9600.0,27145.50,1.000000,27145.500000
1,1,HMR,23017.75,0.0,23200.0,3600.0,0.0,0.0,10400.0,32217.75,1.000000,32217.750000
2,1,MFIR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,1.000000,40234.500000
3,1,MMR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,1.000000,40234.500000
4,1,MMIR,36123.50,0.0,30400.0,5400.0,0.0,0.0,14400.0,46723.50,1.000000,46723.500000
5,1,MFR,22890.00,0.0,28000.0,3600.0,0.0,0.0,12000.0,35290.00,1.000000,35290.000000
6,2,HFR,17545.50,0.0,22080.0,2880.0,0.0,0.0,9600.0,27145.50,0.909091,24677.727273
7,2,HMR,23017.75,0.0,23200.0,3600.0,0.0,0.0,10400.0,32217.75,0.909091,29288.863636
8,2,MFIR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,0.909091,36576.818182
9,2,MMR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,0.909091,36576.818182


,household_class,NPV
0,HFR,183477.081005
1,HMR,217760.539557
2,MFIR,271945.943736
3,MFR,238525.950476
4,MMIR,315805.249280
5,MMR,271945.943736


Saved outputs in ./outputs/
